In [16]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD") # encodes special characters
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(DATABASE_URL)

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM contracts"))
    rows = result.fetchall()

    for row in rows:
        print(row)

(1, 'Acme Corp', 'NDA', 24, 91, 'Passed', datetime.date(2024, 1, 15), 'Germany', 'Data Privacy Policy', 'EU')
(2, 'Zenith Solutions', 'Service Agreement', 36, 74, 'Pending', datetime.date(2023, 11, 20), 'India', 'Vendor Risk Policy', 'APAC')
(3, 'GlobalTech Ltd', 'Partnership', 48, 67, 'Failed', datetime.date(2023, 8, 10), 'United States', 'Financial Compliance Policy', 'North America')
(4, 'Bright Systems', 'Vendor Agreement', 18, 87, 'Passed', datetime.date(2024, 2, 5), 'Singapore', 'Vendor Risk Policy', 'APAC')
(5, 'Nova Innovations', 'Service Agreement', 12, 94, 'Passed', datetime.date(2024, 3, 12), 'United Kingdom', 'HR Compliance Policy', 'UK')
(6, 'Helix Enterprises', 'NDA', 36, 73, 'Pending', datetime.date(2023, 6, 18), 'France', 'Data Privacy Policy', 'EU')
(7, 'Pioneer Holdings', 'Partnership', 24, 86, 'Passed', datetime.date(2024, 4, 2), 'Canada', 'Financial Compliance Policy', 'North America')
(8, 'Apex Consulting', 'Service Agreement', 18, 76, 'Pending', datetime.date(2023

In [45]:
from pymilvus import connections, utility

# Connect to Milvus
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

print("✅ Connected to Milvus")

# Optional: Check existing collections
collections = utility.list_collections()
print("Existing collections:", collections)


✅ Connected to Milvus
Existing collections: ['legal_policy_vectors']


In [13]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),

    FieldSchema(name="contract_id", dtype=DataType.INT64),

    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024),

    FieldSchema(name="contract_type", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="text_chunk", dtype=DataType.VARCHAR, max_length=5000),
]



In [14]:
schema = CollectionSchema(fields, description="Policy Clause Embeddings")

collection = Collection(
    name="legal_policy_vectors",
    schema=schema
)

index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128}
}

collection.create_index(
    field_name="embedding",
    index_params=index_params
)

collection.load()

In [41]:
from sentence_transformers import SentenceTransformer

# Load BGE-M3
model = SentenceTransformer("BAAI/bge-m3")

# Confirm dimension
test_embedding = model.encode("test sentence", normalize_embeddings=True)
print("Embedding dimension:", len(test_embedding))

d:\rag_assignment\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 391/391 [00:00<00:00, 860.12it/s, Materializing param=pooler.dense.weight]                               


Embedding dimension: 1024


In [11]:
def chunk_text(text, chunk_size=800, overlap=150):
    """
    Chunk text with overlap for better retrieval context.
    """
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap

    return chunks

In [15]:
import os

def insert_document(file_path, contract_id, contract_type):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Step 1: Chunk the text
    chunks = chunk_text(text)

    # Step 2: Generate embeddings (normalized for cosine similarity)
    embeddings = model.encode(
        chunks,
        normalize_embeddings=True
    )

    # Step 3: Prepare data in EXACT schema order
    data = [
        [contract_id] * len(chunks),        # contract_id
        embeddings.tolist(),                # embedding
        [contract_type] * len(chunks),      # contract_type
        chunks                              # text_chunk
    ]

    # Step 4: Insert into Milvus
    collection.insert(data)
    collection.flush()

    print(f"✅ Inserted {len(chunks)} chunks for Contract ID {contract_id}")

In [7]:
import torch

def check_device():
    if torch.cuda.is_available():
        print("✅ GPU Available")
        print("GPU Name:", torch.cuda.get_device_name(0))
        return "cuda"
    else:
        print("⚠ GPU Not Available, using CPU")
        return "cpu"

device = check_device()

⚠ GPU Not Available, using CPU


In [8]:
collection_name = "legal_policy_vectors"

if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)
    print("✅ Old collection dropped successfully.")
else:
    print("No existing collection found.")

✅ Old collection dropped successfully.


In [28]:
for i in [23,30]:
    insert_document("D:\\rag_assignment\\rag_data\\rag_data\\partnership\\partnership_v3.txt",i,"Partnership")

✅ Inserted 11 chunks for Contract ID 23
✅ Inserted 11 chunks for Contract ID 30


In [57]:
SYSTEM_PROMPT_EXTRACT = """
You are a legal data extraction assistant.

Your task is to extract structured filters from a user query
and return ONLY valid JSON.

Available database columns:
- vendor_name (string)
- contract_type (must be one of: NDA, Service Agreement, Vendor Agreement, Partnership, General)
- compliance_score (integer)
- audit_status (Passed, Failed, Pending)
- jurisdiction (string)
- region (string)
- duration_months (integer)
- contract_date (date)

Rules:

1. Ignore capitalization differences.
2. Only return JSON.
3. Include only fields explicitly mentioned in the query.
4. Do NOT add extra fields.
5. If nothing relevant is found, return {}.
6. Always strictly use the column names provided above.

Numeric Filtering Rules:

• If query says:
  - "above X", "greater than X", "more than X"
    → use: "compliance_score_min": X

  - "below X", "less than X"
    → use: "compliance_score_max": X

  - "between X and Y"
    → use: "compliance_score_between": [X, Y]

• For duration:
  - "longer than X months"
    → use: "duration_min": X

  - "shorter than X months"
    → use: "duration_max": X

• For relative dates:
  - "last X months"
    → use: "last_n_months": X

• Never output natural language.
• Never explain anything.
• Output must be valid JSON only.

Examples:

Query: Show failed vendor agreements in APAC with compliance score above 70
Output:
{
  "contract_type": "Vendor Agreement",
  "audit_status": "Failed",
  "region": "APAC",
  "compliance_score_min": 70
}

Query: Contracts between 60 and 80 score from last 3 months
Output:
{
  "compliance_score_between": [60, 80],
  "last_n_months": 3
}
"""

In [58]:
import json
from ollama import chat

def extract_filters_from_query(user_query):
    response = chat(
        model='gpt-oss:20b-cloud',
        messages=[
            {'role': 'system', 'content': SYSTEM_PROMPT_EXTRACT},
            {'role': 'user', 'content': user_query}
        ],
        options={
            "temperature": 0
        }
    )

    content = response.message.content.strip()

    # Try parsing JSON safely
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print("⚠ LLM did not return valid JSON:")
        print(content)
        return {}

In [59]:
from sqlalchemy import text
from datetime import datetime, timedelta

def get_contracts_dynamic(filters):
    base_query = "SELECT contract_id FROM contracts WHERE 1=1"
    params = {}

    # 🔹 Text Filters
    if filters.get("vendor_name"):
        base_query += " AND vendor_name ILIKE :vendor_name"
        params["vendor_name"] = f"%{filters['vendor_name']}%"

    if filters.get("contract_type"):
        base_query += " AND contract_type ILIKE :contract_type"
        params["contract_type"] = f"%{filters['contract_type']}%"

    if filters.get("audit_status"):
        base_query += " AND audit_status ILIKE :audit_status"
        params["audit_status"] = f"%{filters['audit_status']}%"

    if filters.get("region"):
        base_query += " AND region ILIKE :region"
        params["region"] = f"%{filters['region']}%"

    if filters.get("jurisdiction"):
        base_query += " AND jurisdiction ILIKE :jurisdiction"
        params["jurisdiction"] = f"%{filters['jurisdiction']}%"

    if filters.get("policy_name"):
        base_query += " AND policy_name ILIKE :policy_name"
        params["policy_name"] = f"%{filters['policy_name']}%"

    # 🔹 Compliance Score Filters
    if filters.get("compliance_score_min"):
        base_query += " AND compliance_score >= :compliance_score_min"
        params["compliance_score_min"] = filters["compliance_score_min"]

    if filters.get("compliance_score_max"):
        base_query += " AND compliance_score <= :compliance_score_max"
        params["compliance_score_max"] = filters["compliance_score_max"]

    # 🔹 Range filter (between)
    if filters.get("compliance_score_between"):
        base_query += " AND compliance_score BETWEEN :score_min AND :score_max"
        params["score_min"] = filters["compliance_score_between"][0]
        params["score_max"] = filters["compliance_score_between"][1]

    # 🔹 Duration Filters
    if filters.get("duration_min"):
        base_query += " AND duration_months >= :duration_min"
        params["duration_min"] = filters["duration_min"]

    if filters.get("duration_max"):
        base_query += " AND duration_months <= :duration_max"
        params["duration_max"] = filters["duration_max"]

    # 🔹 Relative Date Filtering (e.g., last 3 months)
    if filters.get("last_n_months"):
        months = filters["last_n_months"]
        date_threshold = datetime.today() - timedelta(days=30 * months)

        base_query += " AND contract_date >= :date_threshold"
        params["date_threshold"] = date_threshold

    with engine.connect() as conn:
        result = conn.execute(text(base_query), params)
        rows = result.fetchall()

    return rows

In [61]:
query = "Show failed contracts of Pacific Trade Co in APAC"

filters = extract_filters_from_query(query)

print(filters)

{'vendor_name': 'Pacific Trade Co', 'audit_status': 'Failed', 'region': 'APAC'}


In [62]:
def process_user_query(user_query):
    # Step 1: Extract filters using LLM
    filters = extract_filters_from_query(user_query)
    print("🔎 Extracted Filters:", filters)

    # Step 2: Fetch contract IDs from Postgres
    contract_ids = get_contracts_dynamic(filters)
    print("📄 Matching Contracts:", contract_ids)

    return contract_ids

query = "Show contracts in India"
print(process_user_query(query))

🔎 Extracted Filters: {'jurisdiction': 'India'}
📄 Matching Contracts: [(2,), (10,), (12,), (22,), (30,)]
[(2,), (10,), (12,), (22,), (30,)]


In [73]:
from sqlalchemy import text

def get_contract_rows_by_ids(contract_id_rows):

    if not contract_id_rows:
        return []

    # If already list of ints → use directly
    if isinstance(contract_id_rows[0], int):
        contract_ids = contract_id_rows
    else:
        # If list of tuples → extract
        contract_ids = [row[0] for row in contract_id_rows]

    query = text("""
        SELECT *
        FROM contracts
        WHERE contract_id = ANY(:ids)
    """)

    with engine.connect() as conn:
        result = conn.execute(query, {"ids": contract_ids})
        rows = result.fetchall()

    return rows

In [74]:
query = "Show contracts in India"

rows = get_contract_rows_by_ids(process_user_query(query))
for row in rows:
    print(row)

🔎 Extracted Filters: {'jurisdiction': 'India'}
📄 Matching Contracts: [(2,), (10,), (12,), (22,), (30,)]
(2, 'Zenith Solutions', 'Service Agreement', 36, 74, 'Pending', datetime.date(2023, 11, 20), 'India', 'Vendor Risk Policy', 'APAC')
(10, 'Orion Supplies', 'Vendor Agreement', 24, 81, 'Passed', datetime.date(2024, 1, 30), 'India', 'Vendor Risk Policy', 'APAC')
(12, 'IndoLogix Pvt Ltd', 'Service Agreement', 48, 71, 'Pending', datetime.date(2023, 10, 3), 'India', 'Vendor Risk Policy', 'APAC')
(22, 'AsiaTrade Logistics', 'Service Agreement', 12, 90, 'Passed', datetime.date(2024, 5, 5), 'India', 'Vendor Risk Policy', 'APAC')
(30, 'Pacific Alliance Group', 'Partnership', 36, 69, 'Failed', datetime.date(2023, 6, 22), 'India', 'Vendor Risk Policy', 'APAC')


In [65]:
def milvus_search(query_text, matching_contracts, top_k=3):

    # 1️⃣ Convert tuples to flat list
    contract_ids = [row[0] for row in matching_contracts]

    if not contract_ids:
        print("No matching contracts found")
        return []

    # 2️⃣ Create embedding
    query_embedding = model.encode([query_text])

    search_params = {
        "metric_type": "COSINE",
        "params": {"nprobe": 10}
    }

    # 3️⃣ Correct filter expression
    filter_expr = f"contract_id in {contract_ids}"

    results = collection.search(
        data=query_embedding.tolist(),
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        expr=filter_expr,
        output_fields=["contract_id", "contract_type", "text_chunk"]
    )

    retrieved_chunks = []

    for hits in results:
        for hit in hits:
            retrieved_chunks.append({
                "contract_id": hit.entity.get("contract_id"),
                "contract_type": hit.entity.get("contract_type"),
                "text_chunk": hit.entity.get("text_chunk"),
                "score": hit.score
            })

    return retrieved_chunks

In [66]:
from pymilvus import Collection

collection = Collection("legal_policy_vectors")
collection.load()

In [68]:

query = "Show contracts in EU on Intellectual property for Helix Enterprises"

filters = extract_filters_from_query(query)

matching_contracts = get_contracts_dynamic(filters)

vector_results = milvus_search(query, matching_contracts, top_k=5)

for r in vector_results:
    print("Score:", r["score"])
    print("Contract ID:", r["contract_id"])
    print("Text:", r["text_chunk"])
    print("-" * 50)

rows = get_contract_rows_by_ids(process_user_query(query))
for row in rows:
    print(row)

Score: 0.47465652227401733
Contract ID: 6
Text: of:

Unauthorized access to test systems

Exposure of client data

Security vulnerabilities discovered during testing

Cybersecurity incidents affecting confidential environments

The Receiving Party shall notify the Disclosing Party within seventy-two (72) hours of discovery and cooperate in remediation, investigation, and regulatory reporting obligations.

5. Intellectual Property Protection

All software, documentation, test data, and related materials remain the sole property of the Disclosing Party unless otherwise agreed in writing.

Test artifacts, reports, automation frameworks, and documentation created under this Agreement shall be used exclusively for the project scope and shall not be reused for competing commercial advantage without written authorization.

6. Remedies and Ind
--------------------------------------------------
Score: 0.410222589969635
Contract ID: 6
Text: used exclusively for the project scope and shall not be

In [69]:
def build_context(contract_rows, vector_results):
    context = ""

    # Structured metadata section
    context += "STRUCTURED CONTRACT DATA:\n"
    for row in contract_rows:
        context += f"""
Contract ID: {row[0]}
Vendor: {row[1]}
Contract Type: {row[2]}
Duration (months): {row[3]}
Compliance Score: {row[4]}
Audit Status: {row[5]}
Contract Date: {row[6]}
Jurisdiction: {row[7]}
Policy Name: {row[8]}
Region: {row[9]}
-------------------------------------
"""

    # Vector retrieved clauses
    context += "\nRELEVANT CONTRACT CLAUSES:\n"

    for r in vector_results:
        context += f"""
[Contract ID: {r['contract_id']} | Score: {round(r['score'],3)}]
{r['text_chunk']}
-------------------------------------
"""

    return context

In [71]:
from ollama import chat

def generate_answer(user_query, context):

    system_prompt = """
You are a legal compliance assistant.

Answer strictly based on the provided contract data and clauses.
Do NOT hallucinate.
If information is missing, clearly say it is not found in the records.
Provide a professional legal-style summary.
"""

    response = chat(
        model="gpt-oss:20b-cloud",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""
User Query:
{user_query}

Context:
{context}

Provide a structured legal response.
"""}
        ],
    )

    return response.message.content

In [76]:
query = "Show summary about contract with transcontinental corp"

# Step 1: Extract filters
filters = extract_filters_from_query(query)

# Step 2: Structured filtering
matching_contracts = get_contracts_dynamic(filters)

# Step 3: Vector search
vector_results = milvus_search(query, matching_contracts, top_k=5)

# Step 4: Get full rows
contract_rows = get_contract_rows_by_ids(
    [row[0] for row in matching_contracts]
)

# Step 5: Build context
context = build_context(contract_rows, vector_results)

# Step 6: Generate final answer
final_response = generate_answer(query, context)

print("\n💡 FINAL RESPONSE:\n")
print(final_response)


💡 FINAL RESPONSE:

**Contract Summary – Vendor Agreement with TransContinental Corp. (Contract ID 27)**  

| Item | Description |
|-------|-------------|
| **Vendor** | TransContinental Corp |
| **Contract Type** | Vendor Agreement (Cloud‑management/IT services) |
| **Effective Date / Contract Date** | 01 Mar 2024 |
| **Duration** | 18 months |
| **Compliance Score** | 89 % |
| **Audit Status** | Passed |
| **Jurisdiction** | Canada |
| **Policy Covered** | Financial Compliance Policy |
| **Region of Operation** | North America |

---

### 1. Scope of Services
- Cloud infrastructure administration, DevOps management, security configuration, data‑hosting oversight, monitoring, backup, and incident‑response support.  
- Services subject to the Client’s Vendor Risk Policy, Financial Compliance Policy, and applicable regulations in Canada (PIPEDA, provincial privacy laws), the UK (UK GDPR & Data Protection Act 2018), India (DPDPA & IT Act), and any relevant EEA provisions.

### 2. Key Dat